## 1 vs all training

This book assume data is ready based on train_178categories.ipynb

In [ ]:
import os
import sys
import json

import sys


import akagi
print("akagi version:", akagi.__version__)
from akagi.data_sources import RedshiftDataSource, S3DataSource
from akagi.iterator import FileFormat

import numpy as np

import pandas as pd
import glob

In [ ]:
import sys
sys.path.append("../")

In [ ]:
DATA_DIR = ""

BASE_DATA_DIR= os.path.normpath(os.path.join(DATA_DIR, ''))



In [ ]:
#Data dirs {train, validation}
TRAIN_DATA_DIR = os.path.normpath(os.path.join(BASE_DATA_DIR, "train"))
VALID_DATA_DIR = os.path.normpath(os.path.join(BASE_DATA_DIR, "valid"))

In [ ]:
BASE_MODEL_DIR = "./trained_model/inceptionv3/"
BASE_NAME = ""
CODE_EXTRACTOR=""

## Download code extractor from S3

In [ ]:
REGION_NAME = "ap-northeast-1"
DEFAULT_OUTPUT = "text"

os.environ['AWS_DEFAULT_REGION'] = REGION_NAME
os.environ['AWS_DEFAULT_OUTPUT'] = DEFAULT_OUTPUT

!aws s3 

## Training setup

In [ ]:
from models.processor import create_generators

TRAIN_DATAGEN, VALID_DATAGEN = create_generators()

In [ ]:
from one_vs_all import create_multiple_model_trainer

def create_multimodeltrainer():
    return create_multiple_model_trainer(base_model_name = BASE_NAME, basedir = BASE_MODEL_DIR, traingen=TRAIN_DATAGEN, validgen=VALID_DATAGEN, extractor_path=CODE_EXTRACTOR)

In [ ]:
from modelutils import dir2filedict

In [ ]:
multitrainer = create_multimodeltrainer()

In [ ]:
multitrainer.setup_filedict(dir2filedict(TRAIN_DATA_DIR), dir2filedict(VALID_DATA_DIR))

In [ ]:
catkeys = sorted(list(multitrainer.train_files_dict.keys()))

In [ ]:
multitrainer.sample_valid_files(100, seed=123)

In [ ]:
len(multitrainer.valid_files_dict['0'])

## Training

In [ ]:
multitrainer.train_one('38', eachepochs=10)

In [ ]:
multitrainer.remove_checkpoint('38')

In [ ]:
from codeextractor import best_fullmodel_file_to_codemodel_file_by_basename
best_fullmodel_file_to_codemodel_file_by_basename(BASE_MODEL_DIR, BASE_NAME, '38')

In [ ]:
multitrainer.train_one('33', eachepochs=5)

In [ ]:
from codeextractor import best_fullmodel_file_to_codemodel_file_by_basename
best_fullmodel_file_to_codemodel_file_by_basename(BASE_MODEL_DIR, BASE_NAME, '33')

In [ ]:
multitrainer.train_one('49', eachepochs=5)

In [ ]:
best_fullmodel_file_to_codemodel_file_by_basename(BASE_MODEL_DIR, BASE_NAME, '49')

In [ ]:
retraining_keys = ["41", "51", "55", "60", "62", "64", "66", "67", "70", "71", "72", "74", "111", "115", "117", "118", "121", "125", "126", "127", "129"]

In [ ]:
updatekey = "129"

In [ ]:
update_train = os.path.join(TRAIN_DATA_DIR, updatekey)

In [ ]:
update_valid = os.path.join(VALID_DATA_DIR, updatekey)

In [ ]:
s3base = ''

In [ ]:
s3train = os.path.join(os.path.join(s3base, "train"), updatekey)

In [ ]:
s3valid = os.path.join(os.path.join(s3base, "valid"), updatekey)

In [ ]:
! rm -r {update_train}

In [ ]:
! aws s3 cp {s3train} {update_train} --recursive

In [ ]:
! rm -r {update_valid}

In [ ]:
! aws s3 cp {s3valid} {update_valid} --recursive

In [ ]:
import tqdm

In [ ]:
for key in tqdm.tqdm(retraining_keys):
    multitrainer.train_one(key, eachepochs=5)

In [ ]:
"38" + retraining_keys+"49"+

In [ ]:
BACKUPDIR=""

In [ ]:
import shutil

In [ ]:
for key in retraining_keys:
    for f in glob.iglob("{0}_{1}*".format(os.path.join(BASE_MODEL_DIR, "code_" + BASE_NAME), key)):
        shutil.move(f, BACKUPDIR)
        print("move {0}->{1}".format(f, BACKUPDIR))

In [ ]:
from keras import backend as K
import tqdm
from codeextractor import best_fullmodel_file_to_codemodel_file_by_basename

In [ ]:
K.clear_session()

In [ ]:
[best_fullmodel_file_to_codemodel_file_by_basename(BASE_MODEL_DIR, BASE_NAME, key) for key in tqdm.tqdm(retraining_keys)]

In [ ]:
[best_fullmodel_file_to_codemodel_file_by_basename(BASE_MODEL_DIR, BASE_NAME, key) for key in tqdm.tqdm(targetcats)]

In [ ]:
multitrainer.train_one('65', eachepochs=10)

In [ ]:
from codeextractor import best_fullmodel_file_to_codemodel_file_by_basename
best_fullmodel_file_to_codemodel_file_by_basename(BASE_MODEL_DIR, BASE_NAME, '65')

In [ ]:
multitrainer.train_one('0', eachepochs=5)

In [ ]:
# temp 
multitrainer.trainer.set_savepath(multitrainer._model_path('0'))
multitrainer.trainer.remove_checkpoint()

In [ ]:
multitrainer.train_one('1', eachepochs=5)

In [ ]:
catkeys[3:5]

In [ ]:
multitrainer.train_categories(catkeys[3:5], eachepochs=5)

In [ ]:
multitrainer.train_categories(catkeys[60:70], eachepochs=10)

In [ ]:
multitrainer.train_categories(catkeys[70:80], eachepochs=10)

In [ ]:
multitrainer.train_categories(catkeys[80:90], eachepochs=10)

In [ ]:
# finish till catkeys[74], should resume from 75:
catkeys.index('165')

In [ ]:
multitrainer.train_categories(catkeys[75:80], eachepochs=10)

In [ ]:
catkeys.index('17')

In [ ]:
multitrainer.train_categories(catkeys[79:90], eachepochs=10)

In [ ]:
multitrainer.train_categories(catkeys[90:100], eachepochs=10)

## Convert to code model and send result

In [ ]:
targetcats = catkeys[60:100]

In [ ]:
targetcats = {

}

In [ ]:
DIST_DIR=''

In [ ]:
from codeextractor import best_fullmodel_file_to_codemodel_file_by_basename
import tqdm

In [ ]:
[best_fullmodel_file_to_codemodel_file_by_basename(BASE_MODEL_DIR, BASE_NAME, key) for key in tqdm.tqdm(targetcats)]

In [ ]:
os.makedirs(DIST_DIR, exist_ok=True)

In [ ]:
from codeextractor import best_code_path

In [ ]:
best_code_path(BASE_MODEL_DIR, BASE_NAME, '144')

In [ ]:
def link_codemodel_to(targetkey):
    codejson, codeweight = best_code_path(BASE_MODEL_DIR, BASE_NAME, targetkey)
    os.link(codejson, os.path.join(DIST_DIR, os.path.basename(codejson)))
    os.link(codeweight, os.path.join(DIST_DIR, os.path.basename(codeweight)))
    return targetkey

In [ ]:
[link_codemodel_to(key) for key in targetcats]

In [ ]:
S3PATH = "" + DIST_DIR

In [ ]:
# !aws s3 cp --dryrun --exclude "*" --include "code*" {DIST_DIR} {S3PATH} --recursive 
!aws s3 cp --exclude "*" --include "code*" {DIST_DIR} {S3PATH} --recursive

### Misc Evaluation, try the same validaiton set as training time

In [ ]:
BASE_CODE_NAME = "code_" + BASE_NAME

In [ ]:
from one_vs_all import ModelBinder
from codeextractor import DNNCodeExtractor

def ModelBinder_create(base_model_name = BASE_CODE_NAME, basedir = BASE_MODEL_DIR, extractor_path=CODE_EXTRACTOR):
    return ModelBinder(base_model_name, basedir, DNNCodeExtractor.create_from(extractor_path))

###  Whole valid

In [ ]:
from modelutils import dir2filedict
fdict = dir2filedict(VALID_DATA_DIR)
catkeys = sorted(list(fdict.keys()))

### 100 valid

In [ ]:
fdict = multitrainer.valid_files_dict
catkeys = sorted(list(fdict.keys()))

### Predict fdict

In [ ]:
ftuppls = [(key, file) for key in fdict.keys() for file in fdict[key]]
labels = [tup[0] for tup in ftuppls]
paths = [tup[1] for tup in ftuppls]

In [ ]:
len(labels), len(paths)

In [ ]:
binder = ModelBinder_create()

In [ ]:
binder.load_all_models(catkeys)

In [ ]:
%%time
df = binder.predict(paths)

In [ ]:
df['label'] = labels

In [ ]:
VALID_DF_PATH=""

In [ ]:
df.to_pickle(VALID_DF_PATH)

### Valiation score check

In [ ]:
targetkey="60"

In [ ]:
trues = df[df['label'] == targetkey]

In [ ]:
falses = df[df['label'] != targetkey]

In [ ]:
TP = trues[trues[targetkey] > 0.5]

In [ ]:
FP = falses[falses[targetkey] > 0.5]

In [ ]:
TN = falses[falses[targetkey] <= 0.5]

In [ ]:
targetkey="60"

In [ ]:
len(TP), len(TN), len(df)

In [ ]:
(len(TP)+len(TN))/len(df)

In [ ]:
len(TP)/(len(TP) + len(FP))

In [ ]:
len(TP), len(TN), len(df)

In [ ]:
(len(TP)+len(TN))/len(df)

In [ ]:
len(TP)/(len(TP) + len(FP))

In [ ]:
11822/17670

In [ ]:
sum(df['label'] == targetkey)

In [ ]:
targetkey="49"

In [ ]:
df.sort_values(by=targetkey, ascending=False)[0:40]["label"]

In [ ]:
from visualize import plot_image_list

In [ ]:
df.sort_values(by='118', ascending=False)[0:40]['label'][0:10]

In [ ]:
#plot_image_list(df.sort_values(by='49', ascending=False)[0:40]["filepaths"].values)